In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import os
from skimage import io
import tensorflow as tf
import keras
from keras import backend as K 
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.layers import GlobalAveragePooling2D, Dense, Activation, BatchNormalization, Dropout, Input
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
import cv2
from keras.applications.vgg16 import  VGG16, preprocess_input
from skimage.util import view_as_windows 
from scipy.io import loadmat
from scipy.stats import pearsonr, spearmanr
image_dir = '/content/SelectiveEncryption/images/'

In [3]:
!wget http://www.polytech.univ-nantes.fr/autrusseau-f/Databases/SelectiveEncryption/SelectiveEncryption.zip

--2021-09-22 21:07:11--  http://www.polytech.univ-nantes.fr/autrusseau-f/Databases/SelectiveEncryption/SelectiveEncryption.zip
Resolving www.polytech.univ-nantes.fr (www.polytech.univ-nantes.fr)... 193.52.104.60, 2001:660:7220:386:193:52:104:60
Connecting to www.polytech.univ-nantes.fr (www.polytech.univ-nantes.fr)|193.52.104.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49926550 (48M) [application/zip]
Saving to: ‘SelectiveEncryption.zip’

SelectiveEncryption 100%[===================>]  47.61M  8.90MB/s    in 6.4s    

2021-09-22 21:07:18 (7.44 MB/s) - ‘SelectiveEncryption.zip’ saved [49926550/49926550]



In [4]:
!unzip /content/SelectiveEncryption.zip

Archive:  /content/SelectiveEncryption.zip
   creating: SelectiveEncryption/
  inflating: SelectiveEncryption/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/SelectiveEncryption/
  inflating: __MACOSX/SelectiveEncryption/._.DS_Store  
   creating: SelectiveEncryption/images/
  inflating: SelectiveEncryption/images/cimg6013-001-iwind_ec.pgm  
  inflating: SelectiveEncryption/images/cimg6013-001-iwind_nec.pgm  
  inflating: SelectiveEncryption/images/cimg6013-001-trad.pgm  
  inflating: SelectiveEncryption/images/cimg6013-001-trunc.pgm  
  inflating: SelectiveEncryption/images/cimg6013-007-iwind_ec.pgm  
   creating: __MACOSX/SelectiveEncryption/images/
  inflating: __MACOSX/SelectiveEncryption/images/._cimg6013-007-iwind_ec.pgm  
  inflating: SelectiveEncryption/images/cimg6013-007-iwind_nec.pgm  
  inflating: __MACOSX/SelectiveEncryption/images/._cimg6013-007-iwind_nec.pgm  
  inflating: SelectiveEncryption/images/cimg6013-007-trad.pgm  
  inflating: __MACOSX/SelectiveEncrypti

In [5]:
train = pd.read_csv('Path to the training data file/train.csv')
train.head()

,name,ground_truth
0,CIMG7593-001-trad.pgm,0.200000
1,CIMG7593-001-trunc.pgm,0.209524
2,CIMG7593-007-iwind_ec.pgm,0.419048
3,CIMG7593-007-iwind_nec.pgm,0.371429
4,CIMG7593-007-trad.pgm,0.333333


In [6]:
trainedModel = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

x = GlobalAveragePooling2D()(trainedModel.output)

layer = Dense(512,kernel_initializer = 'he_normal')(x)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.2)(layer)

layer = Dense(512, kernel_initializer = 'he_normal')(layer)
layer = BatchNormalization()(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.2)(layer)

outputs = Dense(2)(layer)
model = Model(inputs = trainedModel.input, outputs =outputs)

58900480/58889256 [==============================] - 1s 0us/step


In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#Data pre-processing
Some necessary pre-processing work has been done
<ul>
  <li>Divide all images into Np small patches of size 240 × 240 without overlapping</li>
  <li>Downsample the Np patches of each image to have a size of 224 × 224
  </li>
  <li>Normalize each image patch using the <i>preprocess_input</i> function </li>
</ul>

In [8]:
def train_generator(batch_size):

  while True:
    for pos in range(0, len(train), batch_size):
      temp = train.iloc[pos:pos + batch_size] 
      
      encrypted_batch, ytrain = [], []

      for encimg, y_image in zip(temp.name, temp.ground_truth): 
        encrypted_image = image.load_img(image_dir+encimg)
        encrypted_image = image.img_to_array(encrypted_image)
        
        patches = view_as_windows(encrypted_image,(240, 240, 3),(240, 360, 3)).reshape((-1,240,240,3)) 
   
            
        for patch in patches:
          reshape_patch = cv2.resize(patch, (224,224), interpolation = cv2.INTER_AREA)
          reshape_patch = preprocess_input(reshape_patch)

          encrypted_batch.append(reshape_patch)
          ytrain.append([y_image, y_image])

      encrypted_batch = np.array(encrypted_batch)
      ytrain = np.array(ytrain)

      yield (encrypted_batch, ytrain)

In [9]:
batch_size = 16

for layer in model.layers[:3]:
  layer.trainable = False

for layer in model.layers[3:]:
  layer.trainable = True

model.compile(
    loss= ['mae','mae'],
    loss_weights = [0.4, 0.6],
    optimizer= tf.optimizers.Adam(0.0001)
)

callback = [tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience= 40,
    mode = 'auto',
    min_delta=1e-3,
    restore_best_weights = True,
    verbose =1
)]

history = model.fit(
    train_generator(batch_size),
    epochs = 500,
    verbose = 1,
    batch_size = batch_size,
    callbacks = callback,
    steps_per_epoch= len(train)//batch_size
)

Epoch 1/10
10/10 [==============================] - 65s 1s/step - loss: 0.5025
Epoch 2/10
10/10 [==============================] - 12s 1s/step - loss: 0.3760
Epoch 3/10
10/10 [==============================] - 12s 1s/step - loss: 0.3132
Epoch 4/10
10/10 [==============================] - 12s 1s/step - loss: 0.2757
Epoch 5/10
10/10 [==============================] - 12s 1s/step - loss: 0.2284
Epoch 6/10
10/10 [==============================] - 12s 1s/step - loss: 0.2271
Epoch 7/10
10/10 [==============================] - 12s 1s/step - loss: 0.1878
Epoch 8/10
10/10 [==============================] - 12s 1s/step - loss: 0.1851
Epoch 9/10
10/10 [==============================] - 12s 1s/step - loss: 0.1724
Epoch 10/10
10/10 [==============================] - 12s 1s/step - loss: 0.1711


In [10]:
model.save('The path of the folder where you want to save your trained model/VGG16MAE-04-06-IVC.h5')